In [1]:
import pickle
import numpy as np
from collections import Counter

In [15]:
number_of_games = 100000
filename = "/home/jaoi/master22/pet_for_sale/winning_games_db/4PPO_{}_games.pkl".format(number_of_games)
# with open("/home/jaoi/master22/pet_for_sale/winning_games_db/{}_games.pkl".format(number_of_games), "rb") as fp:
with open(filename, "rb") as fp:
        games = pickle.load(fp)


In [17]:
print(games[0])

[array([18.      ,  0.      ,  0.      ,  0.      ,  0.      ,  1.      ,
         0.      , 18.      ,  3.      ,  3.      , 18.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  1.      ,  2.      , 17.      ,
         8.      ,  2.      , 18.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  1.      ,  4.      , 14.      , 12.      ,  1.      ,
        15.      , 14.      , 13.      , 12.      , 11.      , 10.      ,
         9.      ,  8.      ,  7.      ,  6.      ,  5.      ,  4.      ,
         3.      ,  2.      ,  1.      , 12.      ,  8.      ,  3.      ,
         3.681787], dtype=float32)
 7]


In [70]:
def convert_4_rounds_fs1_obs_to_bin_obs(obs):
    # 4 rounds, so hands also need to have 4 cards ... 
    stack_len = 15
    rounds = 4
    # Should be 9
    coin_numbers = [0, 6, 7, 10, 16, 17, 20, 26, 27]
    
    board_card_numbers = [1,2,3,4,8,11,12,13,14,18,21,22,23,24,28,45,46,47]
    can_bet_numbers = [5,15,25]
    order_numbers = [9,19,29]
    std_number = 48
    last_stack_nr = order_numbers[-1] + stack_len
    bin_rep = []
    onehot_of_stack = np.zeros(stack_len, dtype="float32")
    
    # need to find out if leading by a lot 15-10, medium lead 10-5, or little lead 5-0, tied, behind by a little, medium or a lot
    # this is found through comparing self coins and hand, with the opponents, and finding the opponent closest, or leading by the most
    
    p1_cards = [1,2,3,4,]
    p1_coins = 0
    p2_cards = [11,12,13,14]
    p2_coins = 10
    p3_cards = [21,22,23,24]
    p3_coins = 20
    
    p_values = {}
    p_values["p1_val"] = sum([obs[card] for card in p1_cards]) + obs[p1_coins]
    p_values["p2_val"] = sum([obs[card] for card in p2_cards]) + obs[p2_coins]
    p_values["p3_val"] = sum([obs[card] for card in p3_cards]) + obs[p3_coins]
    lead_enc = np.zeros(7)
    ascending_dict = sorted(p_values.items(), key=lambda x:x[1], reverse=True)
    
    if "p1_val" == ascending_dict[0][0]:
        # p1 is leading
        lead = p_values["p1_val"] - p_values[ascending_dict[1][0]]
    else:
        lead = p_values["p1_val"] - p_values[ascending_dict[0][0]]
        
    if lead > 10:
        # large lead
        lead_enc[0] = 1.0
    elif lead > 5:
        # medium lead
        lead_enc[1] = 1.0
    elif lead > 0:
        # small lead
        lead_enc[2] = 1.0
    elif lead == 0:
        #  even
        lead_enc[3] = 1.0
    elif lead < -10:
        # large trail
        lead_enc[6] = 1.0
    elif lead < -5:
        # medium trail
        lead_enc[5] = 1.0
    elif lead < 0:
        # small trail
        lead_enc[4] = 1.0
        
    
    
    
    for i, num in enumerate(obs):
        if i in coin_numbers:
            bin_rep.extend(list("{0:05b}".format(int(num))))
        elif i in board_card_numbers:
            bin_rep.extend(list("{0:04b}".format(int(num))))
        elif i in can_bet_numbers:
            bin_rep.append(int(num))
        elif i in order_numbers:
            # bin_rep.extend(list("{0:02b}".format(int(num))))
            order = np.zeros(3)
            order[1 - int(num)] = 1.0
            bin_rep.extend(order)
        elif i == std_number:
            if num > (np.std([1,stack_len]) - np.std([1])) / 2:
                bin_rep.append(1)
            else:
                bin_rep.append(0)
            
        else:
            if i == last_stack_nr:
                if int(num) != 0:
                    onehot_of_stack[-int(num)] = 1
                    bin_rep.extend(list(onehot_of_stack))
                else:
                    # onehot_of_stack[-int(num)] = 1
                    bin_rep.extend(list(onehot_of_stack))
            else:
                if int(num) != 0:
                    onehot_of_stack[-int(num)] = 1
    bin_rep.extend(lead_enc)
    # print(onehot_of_stack)
    # print(obs[30:45], len(obs[30:45]))
    bin_rep = np.hstack(bin_rep)
    return bin_rep.astype(np.float32)

In [66]:
convert_4_rounds_fs1_obs_to_bin_obs(games[1359][0]).shape

(152,)

In [71]:
c = Counter()
tsetlinified_games = []
for i, game in enumerate(games):
    tset_game = convert_4_rounds_fs1_obs_to_bin_obs(game[0])
    c[tset_game.shape] += 1
    
    tsetlinified_games.append((tset_game, game[1]))


print(c)
tsetlinified_games = np.array(tsetlinified_games, dtype=object)



Counter({(152,): 707104})


In [72]:
tsetlin_filename = "/home/jaoi/master22/pet_for_sale/winning_games_db/4PPO_{}_tsetlined_leadbit_games.pkl".format(number_of_games)
with open(tsetlin_filename, "wb") as fp:
        pickle.dump(tsetlinified_games, fp)